# 1. Install and Import Dependencies

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [18]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [19]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [20]:
result = model(tokens)

In [21]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [22]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [28]:
r = requests.get('https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*text.*')

In [29]:
results = soup.find_all('div', {'class':regex})
reviews = [result.text for result in results]
reviews[4]

"\n                First of all, I have seen the original FMA and although it was very popular and original, the pacing and conclusion did not sit too well with me. Brotherhood is meant to be a remake of the original, this time sticking to the manga all the way through, but there were people who thought it would spoil the franchise. That myth should be dispelled, as there's only one word to describe this series - EPIC.\n\r\nI admit that as I've seen the original and read the manga, the pacing of Brotherhood seems to start off being VERY fast (I finally got used to the pacing after\n                  ...\nwatching the first fifteen eps or so). Events that took up half a volume of the manga and had spread though a few episodes of the original anime were now shown in just a single episode. However, after trying to look at it from the perspective of someone who's new to FMA (not comparing it to the manga nor the original), I believe that the pacing works and it manages to tell an intriguin

# 5. Load Reviews into DataFrame and Score

In [12]:
import numpy as np
import pandas as pd

In [30]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [31]:
df['review'].iloc[5]

'\n                Fullmetal Alchemist: Brotherhood gets an immense amount of praise in the MAL community.  Now this is just the opinion of one guy. I\'m certainly not the law of the land or anything. However, I personally feel as though calling FMA:B a masterpiece and the champion of all shows is a bit of a stretch. That\'s not to say that I didn\'t enjoy it or that it isn\'t a very solid addition to the ranks of notable shows; I really would like to preface this review by saying that I believe that FMA:B is a very good show and I wouldn\'t be nearly as harsh\n                  ...\non it if it weren\'t for the God status the show has attained in America. I would happily remand "greatest show ever made" to "greatest long-running shonen ever made if you haven\'t seen the 2011 Hunter X Hunter"\n\r\nI\'m not a huge fan of the MAL categorical rating system, as I\'ve mentioned in some of my previous reviews. I oftentimes outright ignore it. However, looking at the categories right now, I f

In [32]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [33]:
sentiment_score(df['review'].iloc[1])

5

In [34]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [35]:
df.tail()

,review,sentiment
4,"\n First of all, I have seen th...",2
5,\n Fullmetal Alchemist: Brother...,4
6,\n Adaptations have long been a...,2
7,\n Antiheroes are often some of the m...,3
8,\n OP characters are a common trope i...,3


In [37]:
df['review'].iloc[7]

'\n          Antiheroes are often some of the most beloved and most talked about characters in their stories. The appeal of these characters can be high and their character brilliant, but what actually makes an antihero work? \n        '